In [2]:
import pandas as pd
import numpy as np
from pyecharts import *
import sklearn as sk

In [3]:
df = pd.read_csv('./data/ccf_offline_stage1_train.csv')
df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
User_id          int64
Merchant_id      int64
Coupon_id        object
Discount_rate    object
Distance         object
Date_received    object
Date             object
dtypes: int64(2), object(5)
memory usage: 93.7+ MB


In [5]:
np.sum(df.isin(['null']),axis=0).apply(lambda x:x/1754883)

User_id          0.000000
Merchant_id      0.000000
Coupon_id        0.399800
Discount_rate    0.399800
Distance         0.060405
Date_received    0.399800
Date             0.557245
dtype: float64

In [6]:
# 领了优惠券但未使用的比例
coupon_used_rate = df[(df.Coupon_id != 'null')&(df.Date == 'null')].count()[0]/1754883
coupon_used_rate

0.55724512688310279

In [7]:
# 领了优惠券并使用了的比例
coupon_unused_rate = df[(df.Coupon_id !='null')&(df.Date !='null')].count()[0]/1754883
coupon_unused_rate

0.042955570257390378

In [8]:
#普通消费比例
normal_rate = df[(df.Coupon_id == 'null')&(df.Date != 'null')].count()[0]/1754883
normal_rate

0.39979987269806588

In [10]:
pie1 = Pie('消费行为类型')
data = [coupon_used_rate,coupon_unused_rate,normal_rate]
index = ['领了优惠券但未使用','领了优惠券并使用','普通消费']
pie1.add('',index,data)
pie1

In [11]:
top10_dscnt_rate = df[(df.Coupon_id != 'null')&(df.Date != 'null')].groupby('Discount_rate')['User_id'].count().sort_values(ascending=False).head(10)
bar1 = Bar('使用最多的10种优惠券')
bar1.add('',top10_dscnt_rate.index,top10_dscnt_rate.values)
bar1

In [12]:
last10_dscnt_rate = df[(df.Coupon_id != 'null')&(df.Date != 'null')].groupby('Discount_rate')['User_id'].count().sort_values(ascending=False).tail(10)
bar2 = Bar('使用最少的10种优惠券')
bar2.add('',last10_dscnt_rate.index,last10_dscnt_rate.values)
bar2

不同额度的优惠券使用人数的多少一定程度上可以反映消费者对该优惠券的消费欲望，但也有可能是因为部分消费券相对稀缺，因为像bar2中50:30这种4折的优惠券肯定是很少的；不妨换种方法，看看不同折扣的优惠券使用率。

In [13]:
coupon_used = df[(df.Coupon_id!='null')&(df.Date!='null')].groupby('Discount_rate').User_id.count()
coupon_total = df[(df.Coupon_id!='null')].groupby('Discount_rate').User_id.count()
coupon_use_rate = (coupon_used/coupon_total).sort_values(ascending=False)
bar3 = Bar('优惠券使用率',width=950,height=400)
bar3.add('',coupon_use_rate.index,coupon_use_rate.values,is_smooth=True,xaxis_interval=1,xaxis_rotate=45)
bar3

从这里看出，优惠券折扣率的高低与消费者是否使用也没有强关联性。

In [14]:
# 使用优惠券的消费者的常活动地点与店铺的距离
coupon_user_distance = df[(df.Coupon_id!='null')&(df.Date!='null')].groupby('Distance').count()
coupon_user_distance

,User_id,Merchant_id,Coupon_id,Discount_rate,Date_received,Date
Distance,,,,,,
0,46852,46852,46852,46852,46852,46852
1,8602,8602,8602,8602,8602,8602
10,2496,2496,2496,2496,2496,2496
2,3485,3485,3485,3485,3485,3485
3,1861,1861,1861,1861,1861,1861
4,1287,1287,1287,1287,1287,1287
5,831,831,831,831,831,831
6,603,603,603,603,603,603
7,430,430,430,430,430,430


In [15]:
# 有优惠券但未使用的消费者的常活动地点与店铺的距离
other_user_distance = df[(df.Coupon_id!='null')&(df.Date=='null')].groupby('Distance').count()
other_user_distance

,User_id,Merchant_id,Coupon_id,Discount_rate,Date_received,Date
Distance,,,,,,
0,301327,301327,301327,301327,301327,301327
1,135269,135269,135269,135269,135269,135269
10,170253,170253,170253,170253,170253,170253
2,77794,77794,77794,77794,77794,77794
3,53083,53083,53083,53083,53083,53083
4,39255,39255,39255,39255,39255,39255
5,30179,30179,30179,30179,30179,30179
6,24055,24055,24055,24055,24055,24055
7,19252,19252,19252,19252,19252,19252


In [30]:
new_df = df[df.Coupon_id!='null']
new_df.count()

User_id          1053282
Merchant_id      1053282
Coupon_id        1053282
Discount_rate    1053282
Distance         1053282
Date_received    1053282
Date             1053282
dtype: int64

In [31]:
def is_used(row):
    if row.Date != 'null':
        return 1
    else:
        return 0

In [32]:
new_df['is_used'] = new_df.apply(is_used,axis=1)

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [33]:
new_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,is_used
1,1439408,4663,11002,150:20,1,20160528,null,0
2,1439408,2632,8591,20:1,0,20160217,null,0
3,1439408,2632,1078,20:1,0,20160319,null,0
4,1439408,2632,8591,20:1,0,20160613,null,0
6,1439408,2632,8591,20:1,0,20160516,20160613,1


In [34]:
new_df.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,is_used
1,1439408,4663,11002,150:20,1,20160528,null,0
2,1439408,2632,8591,20:1,0,20160217,null,0
3,1439408,2632,1078,20:1,0,20160319,null,0
4,1439408,2632,8591,20:1,0,20160613,null,0
6,1439408,2632,8591,20:1,0,20160516,20160613,1


In [27]:
from sklearn import cross_validation
# k折交叉验证，估计模型精度
new_df.corr()

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,User_id,Merchant_id,is_used
User_id,1.000000,0.002253,0.001979
Merchant_id,0.002253,1.000000,0.028828
is_used,0.001979,0.028828,1.000000


In [46]:
bar4 = Bar()
# new_df.groupby('Date').count()['User_id'][:-1]
bar4.add('',new_df.groupby('Date').count()['User_id'][:-1].index,new_df.groupby('Date').count()['User_id'][:-1].values,)
bar4

In [49]:
bar5 = Bar()
# new_df.groupby('Date').count()['User_id'][:-1]
bar5.add('',new_df.groupby('Date_received').count()['User_id'][:-1].index,new_df.groupby('Date_received').count()['User_id'][:-1].values,)
bar5